In [13]:
import googletrans
import numpy as np
import cv2
from flask import Flask, render_template, request, Response, jsonify
#from pipeline import Pipeline

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()        

    def get_frame(self):
        ret, frame = self.video.read()
        while frame is None:
            self.video = cv2.VideoCapture(0)
            ret, frame = self.video.read()

        # DO WHAT YOU WANT WITH TENSORFLOW / KERAS AND OPENCV

        ret, jpeg = cv2.imencode('.jpg', frame)

        return jpeg.tobytes()
    



app = Flask(__name__)

#pipeline = Pipeline()

video_stream = VideoCamera()
#close the cam when the app is closed
@app.teardown_appcontext    
def close_camera(exception):
    video_stream.__del__()
    cv2.destroyAllWindows()

def gen(camera):    
    while True:
        frame = camera.get_frame()
        if frame is not None:
            yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

def apitest():
    return jsonify({"result"})

@app.route('/video_feed')
def video_feed():
     return Response(gen(video_stream),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Mar/2025 22:04:33] "GET / HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "c:\Users\Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\werkzeug\serving.py", line 370, in run_wsgi
    execute(self.server.app)
  File "c:\Users\Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\werkzeug\serving.py", line 331, in execute
    application_iter = app(environ, start_response)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1527, in wsgi_app
    ctx.pop(error)
  File "c:\Users\Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\ctx.py", 

In [4]:
import googletrans
#translate chinese to english
print(googletrans.LANGCODES)
translated=googletrans.Translator().translate("hi, i am a student from university of chinese", dest='zh-tw').text
print(translated)

{'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'azerbaijani': 'az', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-cn', 'chinese (traditional)': 'zh-tw', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'he', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'korean': 'ko', 'kurdish (kurmanji)': 'ku', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', 'latvian': 

In [ ]:
cv2.release()

AttributeError: module 'cv2' has no attribute 'release'